In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Input
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


/opt/miniconda3/envs/GDSvenv/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [12]:
df = pd.read_csv('data/995,000_row_cleaned.csv')

# Remove rows with ivalid values
label_map = {"fake": 1, "satire":1, "conspiracy": 1, "bias": 1, "rumor": 1, "junksci": 1, "hate": 1,  "clickbait": 1,   "political": 1,
             "reliable": 0} #, "unreliable": 1
df = df[df["type"].isin(label_map.keys())]  # Keep only rows with valid labels
print(df["type"].unique())
df["type"] = df["type"].map(label_map)
print(df["type"].unique())
print(df["type"].value_counts())

y = df["type"]

# Using Bag of Words
texts = df["content"]
# vectorizer = CountVectorizer(max_features=10000)  # Limit to top 10,000 words
vectorizer = TfidfVectorizer(
    max_features=10000,  # Increase feature count
)

X_train, X_temp, y_train, y_temp = train_test_split(
    texts, y, 
    test_size=0.2,
    shuffle=False)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, 
    test_size=0.5,
    shuffle=False)

X_train = vectorizer.fit_transform(X_train)  # fit and transform on train
X_val = vectorizer.transform(X_val)          # only transform on val
X_test = vectorizer.transform(X_test)  

/var/folders/z1/hf1tn63x7tl1qjxy1md9xy4w0000gn/T/ipykernel_47491/1965339663.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/995,000_row_cleaned.csv')


['political' 'fake' 'satire' 'reliable' 'conspiracy' 'bias' 'rumor'
 'clickbait' 'hate' 'junksci']
[1 0]
type
1    649783
0    218564
Name: count, dtype: int64


In [13]:
print(f"X_train.shape: {X_train.shape}")
print(f"X_val.shape: {X_val.shape}")
print(f"X_test.shape: {X_test.shape}")
print()

print(f"y_train.shape: {y_train.shape}")
print(f"y_val.shape: {y_val.shape}")
print(f"y_test.shape: {y_test.shape}")

X_train.shape: (694677, 10000)
X_val.shape: (86835, 10000)
X_test.shape: (86835, 10000)

y_train.shape: (694677,)
y_val.shape: (86835,)
y_test.shape: (86835,)


In [14]:
# Try a more complex architecture with residual connections
weight_for_0 = len(y_train) / (2.0 * (y_train == 0).sum())
weight_for_1 = len(y_train) / (2.0 * (y_train == 1).sum())
class_weights = {0: weight_for_0, 1: weight_for_1}
input_layer = Input(shape=(X_train.shape[1],), sparse=True)
x1 = layers.Dense(1500, activation="relu")(input_layer)
x1 = layers.BatchNormalization()(x1)
x1 = layers.Dropout(0.4)(x1)

x2 = layers.Dense(750, activation="relu")(x1)
x2 = layers.BatchNormalization()(x2)
x2 = layers.Dropout(0.3)(x2)

# Residual connection
x3 = layers.concatenate([x1, x2])
x3 = layers.Dense(300, activation="relu")(x3)
x3 = layers.BatchNormalization()(x3)
x3 = layers.Dropout(0.2)(x3)

x4 = layers.Dense(100, activation="relu")(x3)
output = layers.Dense(1, activation="sigmoid")(x4)

NN = tf.keras.models.Model(inputs=input_layer, outputs=output)

#Initialize the NN
NN.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics=[
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.F1Score()],
    weighted_metrics=["f1_score"],  # Weight the F1 metric
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_f1_score',
        mode='max', 
        patience=4,
        restore_best_weights=True
    )
]

#Train the NN
history = NN.fit(
    X_train,y_train,
    validation_data = (X_val,y_val),
    epochs = 10,
    batch_size = 1024,
    callbacks=callbacks,
    class_weight=class_weights
)

Epoch 1/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 149s 218ms/step - f1_score: 0.8589 - loss: 0.2114 - precision_3: 0.9653 - recall_3: 0.9186 - weighted_f1_score: 0.6673 - val_f1_score: 0.8457 - val_loss: 0.1183 - val_precision_3: 0.9678 - val_recall_3: 0.9735 - val_weighted_f1_score: 0.8457
Epoch 2/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 147s 216ms/step - f1_score: 0.8595 - loss: 0.0979 - precision_3: 0.9864 - recall_3: 0.9651 - weighted_f1_score: 0.6685 - val_f1_score: 0.8468 - val_loss: 0.1102 - val_precision_3: 0.9757 - val_recall_3: 0.9689 - val_weighted_f1_score: 0.8468
Epoch 3/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 147s 217ms/step - f1_score: 0.8619 - loss: 0.0479 - precision_3: 0.9944 - recall_3: 0.9815 - weighted_f1_score: 0.6730 - val_f1_score: 0.8472 - val_loss: 0.1186 - val_precision_3: 0.9760 - val_recall_3: 0.9683 - val_weighted_f1_score: 0.8472
Epoch 4/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 147s 216ms/step - f1_score: 0.8656 - loss: 0.0268 - precision_3: 0.9970 - recall_3: 0.9901 - weighted_f1_score: 0.

In [15]:
# Save the model
output_path = "models/nn_alt_classifier2_f1_val_0.9214.keras"
NN.save(output_path) 

In [16]:
# Standard model
# Standard model with corrected tensor flow
weight_for_0 = len(y_train) / (2.0 * (y_train == 0).sum())
weight_for_1 = len(y_train) / (2.0 * (y_train == 1).sum())
class_weights = {0: weight_for_0, 1: weight_for_1}
input_layer = Input(shape=(X_train.shape[1],), sparse=True)
x1 = layers.Dense(1000, activation="relu")(input_layer)
x1 = layers.Dropout(0.4)(x1)
x2 = layers.Dense(500, activation="relu")(x1)
x2 = layers.Dropout(0.2)(x2)
x3 = layers.Dense(100, activation="relu")(x2)
x3 = layers.Dropout(0.2)(x3)
x4 = layers.Dense(25, activation="relu")(x3)
output = layers.Dense(1, activation="sigmoid")(x4)

NN = tf.keras.models.Model(inputs=input_layer, outputs=output)


#Initialize the NN
NN.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics=[
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.F1Score()],
    weighted_metrics=["f1_score"],  # Weight the F1 metric
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_f1_score',
        mode='max', 
        patience=4,
        restore_best_weights=True
    )
]

#Train the NN
history = NN.fit(
    X_train,y_train,
    validation_data = (X_val,y_val),
    epochs = 10,
    batch_size = 1024,
    callbacks=callbacks,
    class_weight=class_weights
)

Epoch 1/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 93s 136ms/step - f1_score: 0.8585 - loss: 0.2340 - precision_4: 0.9650 - recall_4: 0.9024 - weighted_f1_score: 0.6666 - val_f1_score: 0.8497 - val_loss: 0.1465 - val_precision_4: 0.9781 - val_recall_4: 0.9470 - val_weighted_f1_score: 0.8497
Epoch 2/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 96s 141ms/step - f1_score: 0.8676 - loss: 0.1014 - precision_4: 0.9860 - recall_4: 0.9664 - weighted_f1_score: 0.6835 - val_f1_score: 0.8589 - val_loss: 0.1293 - val_precision_4: 0.9817 - val_recall_4: 0.9518 - val_weighted_f1_score: 0.8589
Epoch 3/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 94s 139ms/step - f1_score: 0.8844 - loss: 0.0424 - precision_4: 0.9951 - recall_4: 0.9857 - weighted_f1_score: 0.7162 - val_f1_score: 0.8771 - val_loss: 0.1230 - val_precision_4: 0.9781 - val_recall_4: 0.9645 - val_weighted_f1_score: 0.8771
Epoch 4/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 94s 138ms/step - f1_score: 0.9007 - loss: 0.0222 - precision_4: 0.9976 - recall_4: 0.9925 - weighted_f1_score: 0.7494

In [17]:
# Save the model
output_path = "models/nn_classifier11_f1_val_0.9214.keras"
NN.save(output_path) 

In [ ]:
# ROC curve analysis
from sklearn.metrics import roc_curve, auc

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Learinig curves analysis
# Plot training & validation accuracy/loss values
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

plt.tight_layout()
plt.show()

# Models 
## nn_classifier1_f1_val_0.9328.keras - runtime: 16,6 min
-   1000 (ReLu)(0.4 dropoff), 500 (ReLu)(0.2 dropoff), 100 (ReLu)(0.2 dropoff), 50 (ReLu)
-   5 epochs, 256 bach size
-   val_accuracy: 0.9695 - val_auc: 0.9820 - val_f1_score: 0.9328 - val_loss: 0.1359 - val_precision: 0.9732 - val_recall: 0.9870
### Model evaluation key data
* Test Loss: 0.0917
* Test Accuracy: 0.9685
* Test Precision: 0.9696
* Test Recall: 0.9895
* Test AUC: 0.9901
* Test F1 Score: 0.8937

## nn_classifier2_f1_val_0.9304.keras - runtime: 10,75 min
-   1000 (ReLu)(0.4 dropoff), 500 (ReLu)(0.2 dropoff), 100 (ReLu)(0.2 dropoff), 50 (ReLu)
-   5 epochs, 512 bach size
-   val_accuracy: 0.9690 - val_auc_2: 0.9802 - val_f1_score: 0.9304 - val_loss: 0.1467 - val_precision_2: 0.9744 - val_recall_2: 0.9851
### Model evaluation key data
* Test Loss: 0.0903
* Test Accuracy: 0.9685
* Test Precision: 0.9751
* Test Recall: 0.9836
* Test AUC: 0.9908
* Test F1 Score: 0.8951

## nn_classifier3_f1_val_0.9228.keras - runtime: 10,5 min
-   1000 (ReLu)(0.4 dropoff), 500 (ReLu)(0.2 dropoff), 50 (ReLu)
-   5 epochs, 512 bach size
-   val_accuracy: 0.9703 - val_auc_4: 0.9828 - val_f1_score: 0.9228 - val_loss: 0.1242 - val_precision_4: 0.9752 - val_recall_4: 0.9860
### Model evaluation key data
* Test Loss: 0.0919
* Test Accuracy: 0.9689
* Test Precision: 0.9729
* Test Recall: 0.9866
* Test AUC: 0.9897
* Test F1 Score: 0.8829

## nn_classifier4_f1_val_0.9292
-   5000 (ReLu)(0.4 dropoff),1000 (ReLu)(0.4 dropoff), 500 (ReLu)(0.2 dropoff), 100 (ReLu)(0.2 dropoff), 50 (ReLu)
-   5 epochs, 512 bach size
-   val_accuracy: 0.9697 - val_auc_5: 0.9818 - val_f1_score: 0.9292 - val_loss: 0.1331 - val_precision_5: 0.9750 - val_recall_5: 0.9854
### Model evaluation key data
* Test Loss: 0.0919
* Test Accuracy: 0.9689
* Test Precision: 0.9729
* Test Recall: 0.9866
* Test AUC: 0.9897
* Test F1 Score: 0.8829

## nn_classifier5_f1_val_0.9292 - runtime 10,75
-   Alternate model
-   5 epochs, 512 bach size
-   val_accuracy: 0.9692 - val_auc_6: 0.9815 - val_f1_score: 0.9271 - val_loss: 0.1377 - val_precision_6: 0.9742 - val_recall_6: 0.9856
### Model evaluation key data
* Test Loss: 0.0919
* Test Accuracy: 0.9684
* Test Precision: 0.9737
* Test Recall: 0.9849
* Test AUC: 0.9901
* Test F1 Score: 0.8983

## nn_classifier6_f1_val_0.916 - 16 min
-   1000 (ReLu)(0.4 dropoff), 500 (ReLu)(0.2 dropoff), 100 (ReLu)(0.2 dropoff), 50 (ReLu)
-   5 epochs, 256 bach size

## models/nn_classifier7_f1_val_0.9266.keras - 10 min - NO unreliable
-   1000 (ReLu)(0.4 dropoff), 500 (ReLu)(0.2 dropoff), 100 (ReLu)(0.2 dropoff), 50 (ReLu)
-   5 epochs, 512 bach size

### Model evaluation key data
* Test Loss: 0.0928
* Test Accuracy: 0.9678
* Test Precision: 0.9744
* Test Recall: 0.9827
* Test AUC: 0.9901
* Test F1 Score: 0.8861

## nn_classifier8_f1_val_0.6775 - 5 min - ONLY reliable and fake
-   1000 (ReLu)(0.4 dropoff), 500 (ReLu)(0.2 dropoff), 100 (ReLu)(0.2 dropoff), 50 (ReLu)
-   7 epochs, 1024 bach size

### Model evaluation key data
* Test Loss: 17.5059
* Test Accuracy: 0.3997
* Test Precision: 0.7829
* Test Recall: 0.2741
* Test AUC: 0.5415
* Test F1 Score: 0.6991

## nn_classifier9_f1_val_0.9214 - 11 min - NO unreliable
-   1000 (ReLu)(0.4 dropoff), 500 (ReLu)(0.2 dropoff), 100 (ReLu)(0.2 dropoff), 50 (ReLu)
-   7 epochs, 1024 bach size

### Model evaluation key data
* Test Loss: 0.0991
* Test Accuracy: 0.9665
* Test Precision: 0.9716
* Test Recall: 0.9840
* Test AUC: 0.9896
* Test F1 Score: 0.8901

In [21]:
# Load the model
input_path = "models/nn_classifier11_f1_val_0.9214.keras"
NN = tf.keras.models.load_model(output_path)

In [22]:
# Evaluate on test set
test_results = NN.evaluate(X_test, y_test, verbose=1)
print(f"* Test Loss: {test_results[0]:.4f}")
print(f"* Test Accuracy: {test_results[1]:.4f}")
print(f"* Test Precision: {test_results[2]:.4f}")
print(f"* Test Recall: {test_results[3]:.4f}")
print(f"* Test AUC: {test_results[4]:.4f}")
print(f"* Test F1 Score: {test_results[5]:.4f}")

2714/2714 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - f1_score: 0.9214 - loss: 0.1774 - precision_4: 0.9714 - recall_4: 0.9766 - weighted_f1_score: 0.9214
* Test Loss: 0.1784
* Test Accuracy: 0.9728
* Test Precision: 0.9755
* Test Recall: 0.9234
* Test AUC: 0.9234
* Test F1 Score: 0.9234
